In [1]:
import numpy as np

bin_file_path = r"E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\adc_data_Raw_0.bin"

adc_data = np.fromfile(bin_file_path, dtype=np.int16)
total_values = adc_data.size
total_complex_samples = total_values // 2

print(f"🔢 Total int16 values in file: {total_values}")
print(f"🔀 Total complex (I/Q) samples: {total_complex_samples}")


🔢 Total int16 values in file: 112497840
🔀 Total complex (I/Q) samples: 56248920


In [4]:
import numpy as np
import csv


bin_file_path = r"E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\adc_data_Raw_0.bin"

adc_data = np.fromfile(bin_file_path, dtype=np.int16)
total_values = adc_data.size
total_complex_samples = total_values // 2

print(f"🔢 Total int16 values in file: {total_values}")
print(f"🔀 Total complex (I/Q) samples: {total_complex_samples}")

def bin_to_iq_csv_dynamic(bin_file_path, csv_file_path, num_samples_per_chirp, num_chirps_per_frame, num_antennas):
    print(f"Reading from: {bin_file_path}")

    adc_data = np.fromfile(bin_file_path, dtype=np.int16)
    total_values = adc_data.size

    if total_values % 2 != 0:
        adc_data = adc_data[:-1]  # trim 1 to make even
    total_complex = len(adc_data) // 2

    samples_per_frame = num_samples_per_chirp * num_chirps_per_frame * num_antennas
    num_full_frames = total_complex // samples_per_frame

    usable_values = num_full_frames * samples_per_frame * 2
    adc_data = adc_data[:usable_values]  # trim trailing garbage

    print(f"✔ Using {num_full_frames} full frames out of available data")

    # Reshape to [Frames, Chirps, Samples, Antennas, 2]
    adc_data = adc_data.reshape((num_full_frames,
                                 num_chirps_per_frame,
                                 num_samples_per_chirp,
                                 num_antennas,
                                 2))

    with open(csv_file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Frame Index', 'Chirp Index', 'Sample Index', 'Antenna Index', 'I', 'Q'])

        for frame in range(num_full_frames):
            for chirp in range(num_chirps_per_frame):
                for sample in range(num_samples_per_chirp):
                    for antenna in range(num_antennas):
                        i_val = adc_data[frame, chirp, sample, antenna, 0]
                        q_val = adc_data[frame, chirp, sample, antenna, 1]
                        writer.writerow([frame, chirp, sample, antenna, i_val, q_val])

    print(f"✅ I/Q CSV saved to: {csv_file_path}")


# === Call the function ===
bin_file_path = r"E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\adc_data_Raw_0.bin"
csv_file_path = r"E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\binOutput_IQ.csv"

num_samples_per_chirp = 256
num_chirps_per_frame = 64
num_antennas = 6

bin_to_iq_csv_dynamic(bin_file_path, csv_file_path, num_samples_per_chirp, num_chirps_per_frame, num_antennas)


🔢 Total int16 values in file: 112497840
🔀 Total complex (I/Q) samples: 56248920
Reading from: E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\adc_data_Raw_0.bin
✔ Using 572 full frames out of available data
✅ I/Q CSV saved to: E:\Masters Project\Raw Data\Raw Data Collection\Subject 1 - Run 1 - Sitting - 4 Minutes\binOutput_IQ.csv
